# Fault detection (inference)

One of the main purposes in fault detection tasks is to train model to find faults in cubes which were not included into train dataset. Let's check new cubes. 

In [ ]:
import sys
import glob
import matplotlib.pyplot as plt

sys.path.append('../../..')

from seismiqb import *
from seismiqb.batchflow import FilesIndex, Pipeline, D, B
from seismiqb.batchflow.models.torch import EncoderDecoder, ResBlock
from seismiqb.batchflow.research import Results

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=7

We will use model trained on E cube.

In [ ]:
CROP_SHAPE = (1, 128, 192)
BATCH_SIZE = 64

In [ ]:
INFERENCE_BATCH_SIZE = 32

In [ ]:
def plot_examples(cube_path, model_path, crop_shape, n_images, slide=0):
    dataset = SeismicCubeset(FilesIndex(path=[cube_path], no_ext=True))
    dataset.load(label_dir={
        'amplitudes_01_ETP': '/INPUTS/FAULTS/NPY/*',
        'amplitudes_16_PSDM': '/INPUTS/FAULTS/NPY/*',
        'VZC-cube': '/*faults/*'
    }, labels_class=Fault, transform=True, verify=True)
    dataset.modify_sampler(dst='train_sampler', finish=True, low=0.0, high=1.0)
    
    # Validation pipeline: no augmentations
    val_template = (
        Pipeline()
        .load_model('dynamic', EncoderDecoder, 'model', path=model_path)
        # Load data/masks
        .crop(points=D('train_sampler')(INFERENCE_BATCH_SIZE), shape=CROP_SHAPE)
        .create_masks(dst='masks', width=5)
        .mask_rebatch(src='masks', threshold=0.4, axis=(0, 1))

        .load_cubes(dst='images')
        .scale(mode='q', src='images')
        .predict_model('model',
                       B('images'),
                       fetches='sigmoid',
                       save_to=B('predictions_e_psdm'))
    )
    val_pipeline = val_template << dataset
    
    batch = val_pipeline.next_batch(D('size'), n_epochs=None)
    batch.predictions_e_psdm = (batch.predictions_e_psdm > 0.2).astype(np.float32)
    
    f, ax = plt.subplots(n_images, 2, sharey=True, figsize=(10, 140))
    for b_idx in range(n_images):
        batch.plot_components('images', 'predictions_e_psdm', slide=slide, mode='overlap', idx=b_idx, alpha=[0.9, 0.5], title='_e_psdm', ax=ax[b_idx][0])
        batch.plot_components('masks', 'predictions_e_psdm', slide=slide, mode='overlap', idx=b_idx, title='_e_psdm', ax=ax[b_idx][1])

## PSDM cube

In [ ]:
cube_path = '/data/seismic_data/seismic_interpretation/CUBE_16_PSDM/amplitudes_16_PSDM.hdf5'
plot_examples(cube_path, './research_1d_without_transpose/results/crop_(1, 128, 192)-paths_01_ETP-repetition_0-update_0/2150108386/model_2000',
              CROP_SHAPE, n_images=10)

## VZC cube

In [ ]:
cube_path = '/data/seismic/CUBE_VZS/VZC-cube.hdf5'
plot_examples(cube_path, './research_3/results/crop_(64, 64, 256)-paths_16_PSDM-repetition_0-update_0/3285589856/model_1899', CROP_SHAPE, n_images=16)